# Liquidity Pool Contract Label Maker


In [ ]:
%%capture
### (1) Install dependencies and restart runtime
import os

!pip install -q --upgrade jsonschema==3.2.0
!pip install -q --upgrade web3==5.23.0

os.kill(os.getpid(), 9)  # Hack to restart the runtime after install

In [ ]:
### (2) Define constants

# INSERT Infura free plan API key:
#NODE_ENDPOINT = ""

# Example Addresses
ADDRESSES = {
    "PROTOCOL_A": [        
      "0x94b0a3d511b6ecdb17ebf877278ab030acb0a878",
      "0x9928e4046d7c6513326ccea028cd3e7a91c7590a",
      "0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc",
    ],
    "PROTOCOL_B": [
      "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8",
      "0xcbcdf9626bc03e24f779434178a73a0b4bad62ed",
      "0x7858e59e0c01ea06df3af3d20ac7b0003275d4bf",
    ],
    "PROTOCOL_C": [ 
      "0xceff51756c56ceffca006cd410b03ffc46dd3a58",
      "0x795065dcc9f64b5614c407a6efdc400da6221fb0",
      "0x397ff1542f962076d0bfe58ea045ffa2d347aca0",
    ],
    "PROTOCOL_D": [ 
      "0x0ef1b8a0e726fc3948e15b23993015eb1627f210",
      "0x1f629794b34ffb3b29ff206be5478a52678b47ae",
      "0x6a11f3e5a01d129e566d783a7b6e8862bfd66cca",
    ],
    "PROTOCOL_E": [ 
      "0x75116bd1ab4b0065b44e1a4ea9b4180a171406ed",
      "0xbeabef3fc02667d8bd3f702ae0bb2c4edb3640cc",
      "0x31631b3dd6c697e574d6b886708cd44f5ccf258f",
    ],
}

In [ ]:
### (3) Authenticate with Google Cloud Platform to run BigQuery queries
from google.colab import auth
auth.authenticate_user()

In [ ]:
from web3 import Web3
import requests
import json
from time import sleep

w3 = Web3(Web3.HTTPProvider(NODE_ENDPOINT))

abi_token0 = [
  {
    "constant": True,
    "inputs": [],
    "name": "token0",
    "outputs": [
      {
        "name": "token0",
        "type": "address"
      },
    ],
     "type": "function"
  },
  {
        "constant": True,
        "inputs": [],
        "name": "symbol",
        "outputs": [
            {
                "name": "",
                "type": "string"
            }
        ],
        "payable": False,
        "stateMutability": "view",
        "type": "function"
  }
]

abi_token1 = [
  {
    "constant": True,
    "inputs": [],
    "name": "token1",
    "outputs": [
      {
        "name": "token1",
        "type": "address"
      },
    ],
     "type": "function"
  },
      {
        "constant": True,
        "inputs": [],
        "name": "symbol",
        "outputs": [
            {
                "name": "",
                "type": "string"
            }
        ],
        "payable": False,
        "stateMutability": "view",
        "type": "function"
    }
]

In [ ]:
def token0_token1_data(address):
    # Get pool pair symbols from contract abis:
    # Get token addresses from pool contract address:
    token_0_address=w3.eth.contract(w3.toChecksumAddress(address), abi=abi_token0).functions.token0().call()
    token_1_address=w3.eth.contract(w3.toChecksumAddress(address), abi=abi_token1).functions.token1().call()

    # Create contract instances:
    token_0_contract = w3.eth.contract(w3.toChecksumAddress(token_0_address), abi=abi_token0)
    token_1_contract = w3.eth.contract(w3.toChecksumAddress(token_1_address), abi=abi_token1)

    # Pull token symbols with contract function:
    token0_symbol=token_0_contract.functions.symbol().call()
    token1_symbol=token_1_contract.functions.symbol().call()
    
    return token0_symbol,token1_symbol

In [ ]:
def get_protocol_labels(protocol):
    
    label_list=[]

    for i in range(len(ADDRESSES[protocol])):
    
        # Get contract name from etherscan contract API:        
        URI=f'https://api.etherscan.io/api?module=contract&action=getsourcecode&address='+ADDRESSES[protocol][i]+'&apikey=YourApiKeyToken'
        r=requests.get(URI)
        contract_name=r.json()['result'][0]['ContractName']

        # Get ABI from etherscan API:
        ABI=r.json()['result'][0]['ABI']

        # Interacting with Unisawp V3 pool, UniswapV2Pair and Mooniswap pool ABIs different:    
        if contract_name== 'UniswapV3Pool':
            pool_name=contract_name
            token0_symbol,token1_symbol=token0_token1_data(ADDRESSES[protocol][i])
            
        if contract_name== 'UniswapV2Pair':
            pool_name=w3.eth.contract(w3.toChecksumAddress(ADDRESSES[protocol][i]), abi=ABI).functions.name().call()
            token0_symbol,token1_symbol=token0_token1_data(ADDRESSES[protocol][i])
            
        if contract_name== 'Mooniswap':
            name=w3.eth.contract(w3.toChecksumAddress(ADDRESSES[protocol][i]), abi=ABI).functions.name().call()
            # Pool Name:
            pool_name=name.split("(", 1)[0]
            # Pairs:
            pair=name[name.find('(')+1:name.find(')')]
            token0_symbol=pair.split("-", 1)[0]
            token1_symbol=pair.split("-", 1)[1]
        
        # Create label:
        print(f"{pool_name} : {token0_symbol}-{token1_symbol} Liquidity Pool")
                
        # For parsing of etherscan API:
        sleep(1)    
    return

In [ ]:
# Print all address labels

def print_labels(address_list):

  protocol_names=list(ADDRESSES.keys())

  for i in range(len(protocol_names)):
      print(f'\n{protocol_names[i]} labels:')
      get_protocol_labels(protocol_names[i])
  return

print_labels(ADDRESSES)


PROTOCOL_A labels:
Uniswap V2 : FEI-WETH Liquidity Pool
Uniswap V2 : FEI-TRIBE Liquidity Pool
Uniswap V2 : USDC-WETH Liquidity Pool

PROTOCOL_B labels:
UniswapV3Pool : USDC-WETH Liquidity Pool
UniswapV3Pool : WBTC-WETH Liquidity Pool
UniswapV3Pool : USDC-USDT Liquidity Pool

PROTOCOL_C labels:
SushiSwap LP Token : WBTC-WETH Liquidity Pool
SushiSwap LP Token : SUSHI-WETH Liquidity Pool
SushiSwap LP Token : USDC-WETH Liquidity Pool

PROTOCOL_D labels:
1inch Liquidity Pool  : ETH-1INCH Liquidity Pool
1inch Liquidity Pool  : LDO-0xae7ab96520de3a18e5e111b5eaab095312d7fe84 Liquidity Pool
1inch Liquidity Pool  : ETH-WBTC Liquidity Pool

PROTOCOL_E labels:
Mooniswap V1  : ETH-DAI Liquidity Pool
Mooniswap V1  : ETH-USDT Liquidity Pool
Mooniswap V1  : DAI-USDC Liquidity Pool


# Metamask Fee Analysis

SQL query that returns metrics on fees earned by MetaMask from their swap service during the month of August 2021, only interested in fees paid in ETH and WETH.

Query returns:
- Sum of fees per day
- Average fee per transaction per day
- _Bonus if you have time:_ Cumulative (rolling sum) sum of fees per day

The following addresses were used:
- [Metamask Swap Router](https://etherscan.io/address/0x881d40237659c251811cec9c364ef91dc08d300c)
- [Metamask Swap Contract](https://etherscan.io/address/0x74de5d4fcbf63e00296fd95d33236b9794016631)
- [Metamask Swap Fees]( https://etherscan.io/address/0x11ededebf63bef0ea2d2d071bdf88f71543ec6fb)


- Queried following tables in the bigquery 
[crypto_ethereum](https://console.cloud.google.com/marketplace/product/ethereum/crypto-ethereum-blockchain) dataset
- `transactions`, `traces` and `token_transfers`

In [ ]:
%%bigquery --project solid-topic-331512 df

# Get fees paid in ETH
WITH eth_data AS(
    SELECT
    DATE(block_timestamp) AS date,
    SUM(CAST(value AS FLOAT64)) AS sum_fees_wei,
    AVG(CAST(value AS FLOAT64)) AS average_fees_trans_wei,
    FROM `bigquery-public-data.crypto_ethereum.traces`
    WHERE from_address='0x74de5d4fcbf63e00296fd95d33236b9794016631' #Metamask Swap Contract
    AND to_address='0x11ededebf63bef0ea2d2d071bdf88f71543ec6fb' #Metamask Swap Fee Address
    AND DATE(block_timestamp) > "2021-07-31"
    AND DATE(block_timestamp) < "2021-09-01"
    GROUP BY date
    ORDER BY date
),
# Get fees paid in WETH
weth_data AS(
    SELECT
    DATE(block_timestamp) AS date,
    SUM(CAST(value AS FLOAT64)) AS sum_fees_wei,
    AVG(CAST(value AS FLOAT64)) AS average_fees_trans_wei
    FROM `bigquery-public-data.crypto_ethereum.token_transfers`
    WHERE from_address='0x74de5d4fcbf63e00296fd95d33236b9794016631' #Metamask Swap Contract
    AND to_address='0x11ededebf63bef0ea2d2d071bdf88f71543ec6fb' #Metamask Swap Fee Address
    AND token_address='0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2' #WETH token address
    AND DATE(block_timestamp) > "2021-07-31"
    AND DATE(block_timestamp) < "2021-09-01"
    GROUP BY date
    ORDER BY date
)
# Note that fees in ETH and WETH were combined for each date (denominated in wei)
# ETH and WETH CTEs can be run separately to isolate the fee queries paid in ETH and WETH (denominated in wei)
SELECT e.date,
e.sum_fees_wei+w.sum_fees_wei AS sum_fees_wei,
(e.average_fees_trans_wei+ w.average_fees_trans_wei)/2 AS average_fees_trans_wei,
(SUM(e.sum_fees_wei) OVER (ORDER BY e.date))+(SUM(w.sum_fees_wei) OVER (ORDER BY w.date)) AS rolling_sum_fees_wei
FROM eth_data AS e
JOIN weth_data AS w
ON e.date=w.date

In [ ]:
df

,date,sum_fees_wei,average_fees_trans_wei,rolling_sum_fees_wei
0,2021-08-01,5.680976e+19,8.829481e+15,5.680976e+19
1,2021-08-02,5.332255e+19,1.464987e+16,1.101323e+20
2,2021-08-03,5.432679e+19,1.425913e+16,1.644591e+20
3,2021-08-04,7.653945e+19,1.341540e+16,2.409985e+20
4,2021-08-05,6.416850e+19,1.042480e+16,3.051670e+20
5,2021-08-06,5.456969e+19,1.082958e+16,3.597367e+20
6,2021-08-07,5.224023e+19,9.918263e+15,4.119770e+20
7,2021-08-08,4.566977e+19,7.573783e+15,4.576467e+20
8,2021-08-09,6.098430e+19,1.099566e+16,5.186310e+20
9,2021-08-10,5.123690e+19,9.752026e+15,5.698679e+20
